In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv\data\Database_Vorarlberg_Correction_Progress.xlsx"
df = pd.read_excel(file, "MONUMENT PROTECTION")[3:].fillna('False')

In [ ]:
failed_sites = []
for i, row in df.iterrows():
    try:
        temp_site = Site.objects.get(identifier=row['*SiteID'])
    except ObjectDoesNotExist:
        failed_sites.append(row['*SiteID'])
        temp_site = None
    if temp_site:
        temp_monument, _ = MonumentProtection.objects.get_or_create(
            identifier=row['*Monument Protection ID']
        )
        temp_monument.site_id = temp_site
        
        if row['Public'] == "yes":
            temp_monument.public = True
    
        if row['Unnamed: 9'] != 'False':
            x = None
            for x in row['Unnamed: 9'].split(';'):
                x = x.strip()
                try:
                    item = SkosConcept.objects.get(pref_label=x)
                except ObjectDoesNotExist:
                    item = None
                if item:
                    temp_monument.current_land_use.add(item)
        
        if row['Threats'] != 'False':
            item = None
            x = None
            for x in row['Threats'].split(','):
                x = x.strip()
                try:
                    item = SkosConcept.objects.get(pref_label=x)
                except ObjectDoesNotExist:
                    item = None
                if item:
                    temp_monument.threats.add(item)
        
        if row['Heritage Status'] != 'False':
            if row['Heritage Status'].endswith('no'):
                temp_monument.heritage_status = False

        temp_monument.save()
    

In [ ]:
MonumentProtection.objects.all().delete()